## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### 셀레니움
- 무선청소기에 검색할 URL

    https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A


- 무선 청소기 검색에 사용할 URL(최소화 버전)

    https://search.danawa.com/dsearch.php?query=무선청소기&page=1&limit=120&sort=saveDESC&list=list

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib import parse
from tqdm import tqdm

In [2]:
# 웹드라이버로 크롬 오픈
url = r"https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&originalQuery=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&previousKeyword=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A"
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

In [3]:
html = driver.page_source

soup = BeautifulSoup(html,'html.parser')

In [ ]:
soup

In [45]:
prodItems = soup.select('ul.product_list > li.prod_item')

In [33]:
## id ="product#####" 이런식으로 전개되므로 productItem 이 아니면 패스
prodItems[6].select('p.prod_name > a')[0].text.strip()


'필박스 어몽어스 미니미 지우개 청소기'

In [34]:
#제품 사양 가져오기
prodItems[0].select('div.spec_list')[0].text.strip().replace('\t','')

'핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 150AW / 흡입력: 26000PaPa / [구성] 벽걸이거치 / 브러쉬: 바닥 / 물걸레: 별매 / 침구 / 솔형 / 틈새 / [배터리] 사용시간: 1시간(최대) / 충전시간: 3시간 / 분리형(1개) / [청소] BLDC모터 / 먼지통: 0.5L / [부가] 디스플레이표시 / 헤파필터 / 5단계여과'

In [35]:
#제품 가격 가져오기
prodItems[0].select('input')[1].get('value')

'286610'

In [36]:
#등록월 가져오기
prodItems[0].select('div.prod_sub_meta> dl > dd')[0].text.strip()

'2022.12.'

##### 1번 페이지에서 상품 정보 추출

In [37]:
productData = []

for product in prodItems:
    try:
        product_name = product.select('p.prod_name >a')[0].text.strip()
        product_spec = product.select('div.spec_list')[0].text.strip().replace('\t','')
        product_price = product.select('input')[1].get('value')
        product_reg_mon = product.select('div.prod_sub_meta> dl>dd')[0].text.strip()
        productData.append([product_name,product_spec,product_price,product_reg_mon])
    except:
        pass

In [11]:
import pandas as pd

In [38]:
dfProdData = pd.DataFrame(productData,columns=['제품명','제품 스펙','제품 가격','등록 월'])

In [39]:
dfProdData.tail()

,제품명,제품 스펙,제품 가격,등록 월
115,삼성전자 제트 VS20T9213QDCSW,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,869000,2020.10.
116,삼성전자 제트 VS15T7037P1,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 150W / 소비전력: ...,735900,2020.10.
117,디오바코 V200 LDVCLA200,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 9000Pa / 소비전력: 120W ...,92000,2020.10.
118,TIANHONG 리락쿠마 미니 고급 청소기,"탁상용청소기 / 무선 / 흡입형 / [부가] 색상: 옐로우, 블루 / 크기(가로x세...",8970,2020.10.
119,파나소닉 MC-WDD91,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 100W / 흡입력: 18000Pa ...,179000,2020.10.


##### 여러 페이지 크롤링 하는 방법

- URl

https://search.danawa.com/dsearch.php?query=무선청소기&originalQuery=무선청소기&previousKeyword=무선청소기&checkedInfo=N&volumeType=allvs&page=1&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A

In [18]:
## 검색어랑 ,URL 페이지를 변경하면서 url 생성해주는 함수

def get_search_page_url(keyword,page):
    keyword = parse.quote(keyword)
    url = f'https://search.danawa.com/dsearch.php?query={keyword}&originalQuery={keyword}&previousKeyword={keyword}&checkedInfo=N&volumeType=allvs&page={page}&' + \
        'limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&' +\
    'recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&'+\
        'quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'
    return url


In [47]:
## 상품 정보 추출 함수
def getProdItems(prodItems):
    prodData = []

    for product in prodItems:
        try:
            product_name = product.select('p.prod_name >a')[0].text.strip()
            product_spec = product.select('div.spec_list')[0].text.strip().replace('\t','')
            product_price = product.select('input')[1].get('value')
            product_reg_mon = product.select('div.prod_sub_meta> dl>dd')[0].text.strip()
            prodData.append([product_name,product_spec,product_price,product_reg_mon])
        except:
            pass
    return prodData

In [ ]:
getProdItems(prodItems)

In [15]:
# 검색어에 띄어쓰기가 있다면 링크 URL에 문제가 생길 수도 있으니 urlencode 해주어야한다. -> parse.quote()
get_search_page_url('갤탭 S9',10)

'https://search.danawa.com/dsearch.php?query=%EA%B0%A4%ED%83%AD%20S9&originalQuery=%EA%B0%A4%ED%83%AD%20S9&previousKeyword=%EA%B0%A4%ED%83%AD%20S9&checkedInfo=N&volumeType=allvs&page=10&limit=120&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=10325109&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3138&defaultVaTab=1098867&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A'

In [17]:
total_page = 10

for page in tqdm(range(1,total_page +1)):
    time.sleep(0.5)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]


In [49]:
## 여러 페이지 검색후 크롤링하는 작업

driver = webdriver.Chrome()

# 암묵적으로 맵 자원 로드를 위해서 3초 정도 대기한다.
driver.implicitly_wait(3)

keyword = '무선청소기'
totalPage = 10
# 최종 저장 리스트
prodDataTotal = []

for page in tqdm(range(1,totalPage+1)):
    url = get_search_page_url(keyword,page)
    driver.get(url)
    #페이지 로딩이 완료될 때 까지 5초 대기
    time.sleep(5)
    
    # 상품 정보 추출
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    prodItems = soup.select('ul.product_list > li.prod_item')
    prodItemList = getProdItems(prodItems)

    prodDataTotal += prodItemList

100%|██████████| 10/10 [01:47<00:00, 10.75s/it]


In [51]:
dfProdDataTotal = pd.DataFrame(prodDataTotal)

In [52]:
dfProdDataTotal.tail()

,0,1,2,3
1195,삼성전자 제트 VS20T9213QDCSW,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 200W / 소비전력: ...,869000,2020.10.
1196,삼성전자 제트 VS15T7037P1,핸디스틱청소기 / 무선 / 흡입+물걸레(교체) / 흡입력: 150W / 소비전력: ...,735900,2020.10.
1197,디오바코 V200 LDVCLA200,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 9000Pa / 소비전력: 120W ...,92000,2020.10.
1198,TIANHONG 리락쿠마 미니 고급 청소기,"탁상용청소기 / 무선 / 흡입형 / [부가] 색상: 옐로우, 블루 / 크기(가로x세...",8970,2020.10.
1199,파나소닉 MC-WDD91,핸디스틱청소기 / 무선 / 흡입형 / 흡입력: 100W / 흡입력: 18000Pa ...,179000,2020.10.


In [53]:
dfProdDataTotal.columns =['제품명','제품 스펙','가격','등록월']

In [55]:
dfProdDataTotal.to_excel('./data/다나와_무선청소기_결과.xlsx',index=False)

In [67]:
for i in dfProdDataTotal.index:
    df = dfProdDataTotal.loc[i,'가격']
    try:
        int(df)
    except:
        df = '품절'
    


ValueError: invalid literal for int() with base 10: '//에어메이드 스마트타워 AMC-3501A'

In [65]:
condition = dfProdDataTotal['가격'] == '품절'
dfProdDataTotal[condition]

,제품명,제품 스펙,가격,등록월
